In [2]:
import urllib
import bs4
import pandas
import re
import pandas as pd
from urllib import request

In [3]:
# getting all the links of the pages that dispaly all the restaurants (1,2,3 ..... 169 page) by just looking at the way used to create the links
links=["https://guide.michelin.com/fr/fr/restaurants"]+["https://guide.michelin.com/fr/fr/restaurants/page/"+str(i) for i in range (2,170)]

In [4]:
#requesting all the needed pages to be scraped later
requested_elements=[request.urlopen(url).read() for url in links]

In [5]:
pages = [bs4.BeautifulSoup(request, "lxml") for request in requested_elements]

In [6]:
# finding all the restaurents in one page
restaurents_cards=[page.find_all("div",{"class": "col-md-6 col-lg-6 col-xl-3"}) for page in pages]
restaurents=[]
for i in restaurents_cards:
  restaurents+=i


In [7]:
# we have 3368 restaurents
len(restaurents)

3368

In [8]:
#find all the restaurent pages
restaurents_links=[ restaurent.find("a",{"class":"link"}).get_attribute_list("href")[0] for restaurent in restaurents ] 

In [9]:
restaurents_links=['https://guide.michelin.com/'+ link for link in restaurents_links]
restaurents_links

['https://guide.michelin.com//fr/fr/provence-alpes-cote-dazur/nice/restaurant/la-rotonde-1184636',
 'https://guide.michelin.com//fr/fr/monaco-region/monaco/restaurant/la-montgolfiere-henri-geraci',
 'https://guide.michelin.com//fr/fr/monaco-region/monte-carlo/restaurant/maya-bay',
 'https://guide.michelin.com//fr/fr/monaco-region/fontvieille/restaurant/beefbar208467',
 'https://guide.michelin.com//fr/fr/monaco-region/monte-carlo/restaurant/le-louis-xv-alain-ducasse-a-l-hotel-de-paris',
 'https://guide.michelin.com//fr/fr/monaco-region/monte-carlo/restaurant/la-maree',
 'https://guide.michelin.com//fr/fr/monaco-region/monte-carlo/restaurant/vistamar',
 'https://guide.michelin.com//fr/fr/monaco-region/monaco/restaurant/omer',
 'https://guide.michelin.com//fr/fr/monaco-region/monte-carlo/restaurant/le-blue-bay',
 'https://guide.michelin.com//fr/fr/monaco-region/monte-carlo/restaurant/loga',
 'https://guide.michelin.com//fr/fr/monaco-region/monte-carlo/restaurant/yoshi',
 'https://guide.mi

In [55]:
restaurents_links[402]

'https://guide.michelin.com//fr/fr/ile-de-france/paris/restaurant/lili'

In [48]:

restaurent= bs4.BeautifulSoup(request.urlopen(restaurents_links[18]).read(), "lxml") 


In [76]:
df=pd.DataFrame(columns=["name","address","price","speciality","review","services","phone number","website","latitude","longitude"])

In [62]:
# function to treat the data
def treat_data(restaurent): 
  inf={}
  inf['name']=restaurent.find("h2",{'class':'restaurant-details__heading--title'}).get_text()
  restaurent_details=restaurent.find("ul",{'class':'restaurant-details__heading--list'}).find_all('li')
  inf['address']=restaurent_details[0].get_text()
  price_spec=re.sub("\s+"," ",restaurent_details[1].get_text().replace("\n",'')).split("•")
  # getting the price and the speciality of the restaurant
  if len(re.findall("EUR",price_spec[0]))!=0 :
    if (len(price_spec)==2):
      inf["price"]=price_spec[0]
      inf["speciality"]=price_spec[1]
    elif (len(price_spec)==1):
      inf["price"]=price_spec[0]
  else:
    if (len(price_spec)==1):
      inf["speciality"]=price_spec[0]
  #getting the michelin review
  inf["review"]=restaurent.find("div",{'class':'js-show-description-text'}).get_text().replace("\n",'')
  #getting thre restaurent services
  restautrent_service= [rest.get_text() for rest in restaurent.find("ul",{"class":"row restaurant-details__services--list js-show-more"}).find_all('div')]
  restautrent_service= [" ".join(re.sub("\s+"," ",rest.replace("\n","")).split(" ")[1:]) for rest in restautrent_service ]
  inf["services"]=restautrent_service
  #getting the restaurent information
  restaurentInf=restaurent.find_all('div',{'class':'collapse__block-item'})
  if (len(restaurentInf)==2):
    inf["phone number"]=restaurentInf[0].get_text().replace('\n','')
    inf["website"]=restaurentInf[1].get_text().replace('\n','')
  elif (len(restaurentInf)==1):
    inf["phone number"]=restaurentInf[0].get_text().replace('\n','')
  #getting the coordination of restaurent from the query link used by the google map api
  restaurentLink=restaurent.find('div',{'class':'google-map__static'}).find("iframe",{'frameborder':'0'}).get_attribute_list("src")[0]
  inf["latitude"]=re.findall('=\d+.\d+|=-\d+.\d+',restaurentLink)[0][1:]
  inf["longitude"]=re.findall(',\d+.\d+|,-\d+.\d+',restaurentLink)[0][1:]
  return inf


In [77]:
#collecting all the restaurent_data
for i in range(len(restaurents_links)):
  print(i)
  restaurent=bs4.BeautifulSoup(request.urlopen(restaurents_links[i]))
  inf=treat_data(restaurent)
  df=df.append(inf,ignore_index=True)


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [78]:
df

,name,address,price,speciality,review,services,phone number,website,latitude,longitude
0,La Rotonde,"37 promenade des Anglais, Nice, 06000, France",49 - 89 EUR,Cuisine méditerranéenne,La brasserie – en forme de rotonde – du palace...,"[Restaurant non fumeur , Repas en terrasse , A...",+33 4 93 16 64 11,www.hotel-negresco-nice.com,43.6942999,7.2579981
1,La Montgolfière-Henri Geraci,"16 rue Basse, Monaco, 98000, France",31 - 70 EUR,Cuisine moderne,"Dans une ruelle piétonne du rocher, à deux pas...","[Restaurant non fumeur , Repas en terrasse , A...",NaN,NaN,43.7310300,7.4225990
2,Maya Bay,"24 avenue Princesse-Grace, Monte-Carlo, 98000,...",18 - 180 EUR,"Cuisine thaïlandaise, Cuisine japonaise","Dans un même lieu, un restaurant japonais et u...","[Accès handicapés , Restaurant non fumeur , Re...",NaN,NaN,43.7488000,7.4354800
3,Beefbar,"42 quai Jean-Charles-Rey, port de Fontvieille,...",45 - 130 EUR,"Spécialités de Viandes, Spécialités de Grilla...",Sur les quais du port de plaisance de Fontviei...,"[Vue , Restaurant non fumeur , Air conditionné...",+377 97 77 09 29,www.monaco.beefbar.com,43.7278400,7.4215800
4,Le Louis XV - Alain Ducasse à l'Hôtel de Paris,"Place du Casino, Monte-Carlo, 98000, France",180 - 380 EUR,Cuisine méditerranéenne,"Difficile de présenter le Louis XV, sans évoqu...","[Accès handicapés , Voiturier , Restaurant non...",NaN,NaN,43.7392006,7.4273302
...,...,...,...,...,...,...,...,...,...,...
3363,Monsieur P,"8 place des Célestins, Lyon, 69002, France",32 - 98 EUR,Cuisine moderne,"Nouvelle adresse pour Monsieur P, qui a eu l'a...","[Restaurant non fumeur , Salon particulier , C...",NaN,NaN,45.7595410,4.8317011
3364,Le Bastion,"50 boulevard de la Promenade, Lagrasse, 11220,...",21 - 60 EUR,Cuisine moderne,Cet ancien bastion médiéval du huitième siècle...,"[Restaurant non fumeur , Repas en terrasse , C...",NaN,NaN,43.0891229,2.6196977
3365,Quatre Saisons,"10 rue Jean-Baptiste-Champeil, Aurillac, 15000...",34 - 75 EUR,Cuisine moderne,Sincère et bien tournée : telle est la cuisine...,"[Restaurant non fumeur , Air conditionné , Car...",NaN,NaN,44.9298600,2.4468850
3366,Le Relais de Beaufort,"RN 2, Beaufort, 59330, France",NaN,NaN,Une auberge contemporaine ornée de sculptures ...,"[Accès handicapés , Restaurant non fumeur , Re...",NaN,NaN,50.2101930,3.9489885
